### 서울시 주유소 가격 정보를 갖고 있는 엑셀 파일 자동 다운로드
- XPATH 사용

In [ ]:
# XPATH 
# - XML Path Language
# - XML 문서의 특정 요소 속성에 접근하기 위한 경로를 지정하는 언어
# - XPATH 식 : 경로식 사용, 수학함수 기타 확장 가능한 표현(기호) 사용

#### Opinet 사이트  
https://www.opinet.co.kr/user/main/mainView.do  
주유소의 가격을 비교하는 사이트  


In [1]:

# 메뉴에서 [싼 주유소 찾기] / [지역별]
# 이 페이지부터 시작
# https://www.opinet.co.kr/searRgSelect.do
# 접속한 지역으로 자동 설정됨
# 지역을 [서울] / [강남구] 로 선택

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
from urllib.request import urlopen
import pandas as pd
import bs4

In [3]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 객체(모듈)

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
# (1) webdriver 객체 생성
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [5]:
# 페이지 접속
url = 'https://www.opinet.co.kr/searRgSelect.do'
driver.get(url)

# 다른 지역에서 접속 시
# 페이지 열리면
# 서울 강남구 선택

In [ ]:
# XPATH 값 복사하기
# 개발자도구 열고 [강남구] select 태그 선택하고
# 오른쪽 코드에서 선택된 부분 우클릭 / [Copy}] / [Copy Xpath]
# //*[@id="SIGUNGU_NM0"]

In [8]:
# Xpath : //*[@id="SIGUNGU_NM0"]
# Xpath 사용
xpath = '//*[@id="SIGUNGU_NM0"]'
gu_list_raw = driver.find_element(By.XPATH, xpath)
# gu_list_raw 

gu_list = gu_list_raw.find_elements(By.TAG_NAME, 'option')
# gu_list # element
# <selenium.webdriver.remote.webelement.WebElement (session="452699f3691a6dfdfe37e8d9d11b47cf", element="47C2966F623140A7A14C409FB28E3927_element_476")>

In [9]:
# element에서 속성값 추출 
# <select> 태그 내의 <option> 태그의 value 속성
gu_name_list = [ opt.get_attribute('value')  for opt in gu_list ]
gu_name_list.remove('')
gu_name_list

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [10]:
# 테스트
# gu_name_list에 이쓴 구 이름에 해당되는 페이지가 동적으로 바뀌는지 확인
# 첫 번째 구 이름(강남구)으로 확인
sigungu_sel = driver.find_element(By.ID, 'SIGUNGU_NM0') # 첫 번째 강남구

# option 태그의 value 값을 가져와서 
# send_keys(구이름) 호출하면서 전달 (입력 값 전달)
# 입력값에 해당되는 화면으로 전환되고
# 전환된  화면에 바뀐 데이터 출력 

sigungu_sel.send_keys(gu_name_list[2]) # 강북구 값 전달 
# 인덱스 숫자 변경하면 지도에서 해당구의 지도로 자동 변경되는지 확인
# send_keys() : 값을 전달해서 웹 브라우저 내용을 동적으로 변경하는 메소드

In [ ]:
# 선택자로 선택
# 기능 수행 : 다음 페이지로 전환해서 페이지 내의 데이터 추출
# 기능 수행 메소드 2가지 방법
# (1) send_keys()
# (2) click()

In [ ]:
# 엑셀 파일  자동 저장
# 왼쪽 주유 가격 맨 아래에 [엑셀저장] 버튼을 자동 실행시켜서
# 모든 구의  주유  정보를 포함하는 엑셀 파일을 자동으로 다운로드

In [12]:
# 테스트 : 1개 파일 다운로드 되는지 확인
# [엑셀저장] 버튼의 XPATH 복사해서 사용
xpath = '//*[@id="glopopd_excel"]'
excel_path = driver.find_element(By.XPATH, xpath)
excel_path.click()

In [13]:
driver.close()

In [ ]:
### 서울 시 모든 구의 주유가격 정보 파일 자동 다운로드

In [14]:
# (1) webdriver 객체 생성
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [15]:
# 페이지 접속
url = 'https://www.opinet.co.kr/searRgSelect.do'
driver.get(url)

# 다른 지역에서 접속 시
# 페이지 열리면
# 서울 강남구 선택

In [ ]:
# 진행 상황 표시 (프로그레스 바 출력하) : tqdm 패키지 사용
# !pip install tqdm

In [16]:
from tqdm.notebook import tqdm
import time

In [ ]:
for gu in tqdm(gu_name_list) : # 구별 페이지에서 데이터 추출하는 진행상황 표시 (프로그레스 바 출력)
    # 구별로 '구이름' 값 전달하고(입력) 동적 페이지 실행
    sigungu_sel = driver.find_element(By.XPATH, '//*[@id="SIGUNGU_NM0"]')
    sigungu_sel.send_keys(gu)

    time.sleep(2)  # 2초 딜레이

    # [엑셀저장] 버튼 클릭 실행
    xpath = '//*[@id="glopopd_excel"]'
    excel_path = driver.find_element(By.XPATH, xpath)
    excel_path .click()

    time.sleep(2)  # 2초 딜레이
    
    

  0%|          | 0/25 [00:00<?, ?it/s]

In [12]:
driver.close()